In [1]:
#!pip install --quiet ipython-autotime
%load_ext autotime

time: 0 ns (started: 2022-02-21 09:39:16 +05:30)


In [2]:
import transformers
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

time: 15.3 s (started: 2022-02-21 09:39:16 +05:30)


In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-6-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-6-6")

to_tokenize = """In a traditional world, most companies resolve support issues manually. In this approach, customers or employees request a ticket through e-mail, ITSM portal, or call. This ticket is picked by a Level 1 technician based on availability after a long wait. During this time, customer or employee is frustrated and often unproductive waiting for their issues to be resolved. If the Level 1 technician can not resolve the issue, it will be escalated to Level 2, and the cycle of wait time and explanation restarts, increasing loss in productivity and frustration. If the Level 2 technical cannot resolve, it goes to Level 3 and then to the Supervisor / Manager. This approach gives the most human touch possible. It is not productive and efficient in this current world where time is gold, and no one wants to wait even for a few minutes, and surprisingly, customers and employees have to wait for minutes, hours, or in few cases - even days."""

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer,framework="tf")

time: 52.2 s (started: 2022-02-21 09:39:31 +05:30)


In [20]:
import pickle
pickle.dump(summarizer, open('model.pkl', 'wb'))

time: 3min 5s (started: 2022-02-21 10:00:07 +05:30)


In [ ]:
pickled_model = pickle.load(open('model.pkl', 'rb'))
pickled_model.predict(to_tokenize)

In [4]:
summarized = summarizer(to_tokenize, min_length=75)

time: 10.9 s (started: 2022-02-21 09:40:23 +05:30)


In [5]:
# Print summarized text
print(summarized)

[{'summary_text': ' In a traditional world, most companies resolve support issues manually . In this approach, customers or employees request a ticket through e-mail, ITSM portal, or call . This ticket is picked by a Level 1 technician based on availability after a long wait . This approach gives the most human touch possible. It is not productive and efficient in this current world where time is gold,'}]
time: 0 ns (started: 2022-02-21 09:40:34 +05:30)


In [6]:
import pke
import string
from nltk.corpus import stopwords
import spacy

#keyword extraction
def multipartite_keyword(text):
    out=[]
    try:
        # 1. create a MultipartiteRank extractor.
        extractor = pke.unsupervised.MultipartiteRank()

        # 2. load the content of the document.
        extractor.load_document(text)

        # 3. select the longest sequences of nouns and adjectives, that do
        #    not contain punctuation marks or stopwords as candidates.
        pos = {'NOUN', 'PROPN', 'ADJ'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extractor.candidate_selection(pos=pos, stoplist=stoplist)

        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.74,
                                      method='average')

        # 5. get the 10-highest scored candidates as keyphrases
        keyphrases = extractor.get_n_best(n=10)

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out
    

time: 8.67 s (started: 2022-02-21 09:40:34 +05:30)


In [7]:
#sumary text sentence tokenization
summary=summarized[0]['summary_text']

time: 0 ns (started: 2022-02-21 09:40:43 +05:30)


In [8]:
#keyword mapping
from flashtext import KeywordProcessor


def get_keywords(originaltext,summarytext):
    keywords =  multipartite_keyword(originaltext)
    print ("keywords unsummarized: ",keywords)
    keyword_processor = KeywordProcessor()
    for keyword in keywords:
        keyword_processor.add_keyword(keyword)

    keywords_found = keyword_processor.extract_keywords(summarytext)
    keywords_found = list(set(keywords_found))
    print ("keywords_found in summarized: ",keywords_found)

    important_keywords =[]
    for keyword in keywords:
        if keyword in keywords_found:
            important_keywords.append(keyword)

    return important_keywords

time: 203 ms (started: 2022-02-21 09:40:43 +05:30)


In [9]:
imp_keywords = get_keywords(to_tokenize,summary)
print (imp_keywords)

keywords unsummarized:  ['level', 'customers', 'time', 'employees', 'support issues', 'traditional world', 'approach', 'frustrated', 'productivity', 'technician']
keywords_found in summarized:  ['approach', 'employees', 'support issues', 'traditional world', 'customers', 'technician', 'level', 'time']
['level', 'customers', 'time', 'employees', 'support issues', 'traditional world', 'approach', 'technician']
time: 1.64 s (started: 2022-02-21 09:40:43 +05:30)


In [10]:
#called pretrained question generator model
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

time: 21.3 s (started: 2022-02-21 09:40:45 +05:30)


In [21]:
pickle.dump(question_model, open('que_generator.pkl', 'wb'))
# pickled_model = pickle.load(open('que_generator.pkl', 'rb'))
# pickled_model(to_tokenize)

time: 1min 25s (started: 2022-02-21 10:13:30 +05:30)


In [24]:
qg_model = pickle.load(open('que_generator.pkl', 'rb'))

time: 40.8 s (started: 2022-02-21 10:17:08 +05:30)


In [25]:
def get_question(context,answer,model,tokenizer):
    text = "context: {} answer: {}".format(context,answer)
    encoding = tokenizer.encode_plus(text,max_length=384, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    outs = qg_model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


    dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]


    Question = dec[0].replace("question:","")
    Question= Question.strip()
    return Question

time: 188 ms (started: 2022-02-21 10:18:05 +05:30)


In [26]:
ls=[]
for answer in imp_keywords:
    ques = get_question(summary,answer,question_model,question_tokenizer)
    q=ques
    a=answer.capitalize()
    b=[q,a]
    ls.append(b)

time: 30.9 s (started: 2022-02-21 10:18:20 +05:30)


In [17]:
ls

[["What level of technician picks a customer's ticket?", 'Level'],
 ['Who requests a ticket through e-mail, ITSM portal or call?', 'Customers'],
 ['In a modern world where what is gold?', 'Time'],
 ['Customers or what other group may request a ticket through e-mail, ITSM portal, or call?',
  'Employees'],
 ['In a traditional world, most companies resolve what manually?',
  'Support issues'],
 ['In what type of world do most companies resolve support issues manually?',
  'Traditional world'],
 ['What approach gives the most human touch possible?', 'Approach'],
 ['What level 1 person picks up a customer support ticket?', 'Technician']]

time: 15 ms (started: 2022-02-21 09:47:46 +05:30)
